# 11. Household power consumption

Dans ce notebook nous étudions la pertinence du clustering sur une série temporelle de consommation d'électricité d'un foyer.

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

## 11.1 Chargement du dataset

Chargeons notre dataset puis affichons quelques lignes et caractéristiques :

In [ ]:
dataset_url = '/data/household_power_consumption.csv'
df = pd.read_csv(dataset_url, index_col=0)
df.index = pd.to_datetime(df.index)
print(df.shape)
df.head()

In [ ]:
df.tail()

In [ ]:
df.info()

In [ ]:
df.describe()

Ce dataset contient la consommation en électricité d'un foyer toutes les minutes de 2006 à 2010.

Vérifions si notre dataset contient des valeurs manquantes :

In [ ]:
df[df.Global_active_power.isna()]

Notre dataset contenant effectivement des valeurs manquantes, nous devons les remplacer. Nous appliquons simplement la fonction [`fillna`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html) sur le dataset avec le paramètre `method='bfill'` pour reprendre la dernière valeur horaire présente avant la valeur manquante (une méthode plus élaboré reposant sur une interpolation serait probablement utile, il faudrait aussi probablement analyser la répartition de ces valeurs manquantes comme par exemple analyser la durée des plages manquantes) :

In [ ]:
df = df.fillna(method='bfill')

## 11.2 Transformation du dataset

L'objectif que nous nous fixons est de vérifier si nous pouvons mettre en évidence des profils de consommation quotidien.

Commencons par créer une nouvelle dataframe contenant la somme des consommations par heure avec la méthode [`resample`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.resample.html) en précisant le paramètre `H`. Nous appliquons ensuite la méthode `sum` à la suite du resample pour calculer la consommation horaire :

In [ ]:
df_hourly = df.resample('H').sum()

In [ ]:
df_hourly.head()

Transformons notre dataframe de consommation horaire pour que son index ne soit plus que la date du jour et qu'elle possède une colonne indiquant l'heure :

In [ ]:
df_hourly['hour'] = df_hourly.index.hour
df_hourly.index = df_hourly.index.date

Puis nous appliquons la méthode `pivot` sur notre dataframe avec comme paramètre `columns='hour'`. Nous obtenons une dataframe contenant une ligne par jour et une colonne par heure de la journée :

In [ ]:
df_pivot = df_hourly.pivot(columns='hour')
df_pivot.head()

Les colonnes sont un multi-index qui sera génant pour l'affichage. Nous pouvons les transformer pour ne conserver que les heures (sans le premier niveau de l'index qui n'indique que `Global_active_power`) :

In [ ]:
df_pivot.columns = df_pivot.columns.get_level_values(1)
df_pivot.head()

Vérifions si notre dataframe contient des valeurs manquantes :

In [ ]:
df_pivot[df_pivot.isna().any(axis='columns')]

Ces valeurs manquantes correspondant au premier jour des mesures, nous pouvons supprimer ce jour :

In [ ]:
df_pivot = df_pivot.dropna()

La cellule suivante permet de visualiser les consommation horaires pour tous les jours de notre dataframe (chaque tracé correspond à une journée) :

In [ ]:
df_pivot.T.plot(figsize=(13,8), legend=False, color='blue', alpha=0.02)

## 11.3 Clustering

Appliquons une série de K-means pour 2 à 31 clusters. Pour chaque nombre de clusters, nous stockons l'inertie et calculons le silhouette score, puis affichons l'elbow curve pour le silouhette score et l'inertie :

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import silhouette_score

silhouette_scores = []
inertia = []
n_cluster_list = np.arange(2,31).astype(int)

X = df_pivot.values.copy()
    
# Very important to scale!
sc = MinMaxScaler()
X = sc.fit_transform(X)

for n_cluster in n_cluster_list:
    
    kmeans = KMeans(n_clusters=n_cluster)
    cluster_found = kmeans.fit_predict(X)
    silhouette_scores.append(silhouette_score(X, kmeans.labels_))
    inertia.append(kmeans.inertia_)

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(n_cluster_list, silhouette_scores)
plt.title('Silhouette scores per cluster size')
plt.xlabel('Number of clusters')
plt.ylabel('Silhouette score')
plt.show()

In [ ]:
plt.figure(figsize=(15, 6))
plt.plot(n_cluster_list, inertia)
plt.title('Inertia per cluster size')
plt.xlabel('Number of clusters')
plt.ylabel('Inertia')
plt.show()

L'elbow curve avec le silouhette score fait ressortir qu'il faut 3 clusters pour séparer au mieux les jours de notre dataset (nous avons un silouhette score plus élevé pour 2 clusters, mais il est préférable de mieux séparer nos données).

Appliquons maintenant un K-means avec le nombre de clusters voulu et affichons le nombre de jours par cluster :

In [ ]:
kmeans = KMeans(n_clusters=3)
clusters = kmeans.fit_predict(X)
pd.Series(clusters).value_counts()

Créons une nouvelle colonne sur notre dataframe pivot pour associer le numéro de cluster à chaque jour :

In [ ]:
df_pivot['cluster'] = clusters
df_pivot.head()

Calculons le profil médian et moyen des jours de nos clusters :

In [ ]:
clusters_median = df_pivot.groupby('cluster').median().values
clusters_mean = df_pivot.groupby('cluster').mean().values
# clusters_mean = sc.inverse_transform(kmeans.cluster_centers_)  # alternative using the scaler inverse transform

Finalement, nous pouvons afficher les profils des jours median (ou moyen) de nos clusters ainsi que tous les jours de chaque cluster. Dans la cellule suivante, nous choisissons une palette de couleur pour nos clusters puis nous affichons chaque jour des clusters avec une forte transparence ainsi que le profil médian pour ces clusters. Pour choisir d'afficher le profil moyen, nous pouvons remplacer `clusters_median` par `clusters_mean`.

In [ ]:
fig, ax= plt.subplots(1,1, figsize=(18,10))
colors = plt.cm.jet(np.linspace(0,1,kmeans.n_clusters))
for cluster in range(kmeans.n_clusters):
    df_pivot[df_pivot.cluster == cluster].T.plot(
        ax=ax, legend=False, alpha=0.01, color=colors[cluster], label= f'Cluster {cluster}'
    )
    plt.plot(clusters_median[cluster], color=colors[cluster], alpha=1, ls='--')
ax.set_xticks(np.arange(1,25))
ax.set_ylabel('kilowatts')
ax.set_xlabel('hour')
plt.show()

## 11.4 Pour aller plus loin

Avec 3 clusters, nous avons mis en évidence 3 profils de consommation quotidien, probablement liés à des jours travaillés, de présence (vacances, jours féries ou week-end) et d'absence (vacances ?).

Pour aller plus loin, il serait utile de vérifier si notre hypothèse sur les profils moyen obtenus correspondent bien à des jours travaillés, de présence ou d'absence en utilisant le calendrier locale du dataset (les données ont été collectées en France). Vous pouvez aussi essayer de séparer les jours sur 4 clusters et d'expliquer le nouveau cluster obtenu.